In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

### Load the data

In [3]:
# Data Loading
import pandas as pd
import os

path = r'C:\Users\TA\TextAnalysis_News'

ada_derena_news = pd.read_csv(os.path.join(path , 'ada_derena_news.csv'))
ada_derana_sports_news = pd.read_csv(os.path.join(path , 'ada_derana_sports_news.csv'))
ceylon_today_news = pd.read_csv(os.path.join(path , 'ceylon_today_news.csv'))
colombo_gazette_news = pd.read_csv(os.path.join(path, 'colombo_gazette_news.csv'))
daily_mirror_news = pd.read_csv(os.path.join(path , 'daily_mirror_news.csv'))
lanka_express_news = pd.read_csv(os.path.join(path , 'lanka_express_news.csv'))
news_1st = pd.read_csv(os.path.join(path , 'news_1st.csv'))
news_wire = pd.read_csv(os.path.join(path, 'news_wire.csv'))
sri_lanka_mirror_news = pd.read_csv(os.path.join(path, 'sri_lanka_mirror_news.csv'))
the_morning_news = pd.read_csv(os.path.join(path, 'the_morning_news.csv'))

In [ ]:
lanka_news_sources = [ada_derena_news, ada_derana_sports_news, ceylon_today_news, colombo_gazette_news, daily_mirror_news, lanka_express_news, news_1st, news_wire, sri_lanka_mirror_news, the_morning_news]

In [ ]:
# Function to rename column names
def rename_columns(dataframes):
    for source_df in dataframes:
        # Rename the first column to 'articles'
        source_df.rename(columns={source_df.columns[0]: 'Articles'}, inplace=True)

In [ ]:
# Call the function to rename columns
rename_columns(lanka_news_sources)

In [ ]:
source_names = ["Ada Derena News", "Ada Derana Sports News", "Ceylon Today News", "Colombo Gazette News", "Daily Mirror News", "Lanka Express News", "News 1st", "News Wire", "Sri Lanka Mirror News", "The Morning News"]
def add_source_name_column(dataframes, source_names):
    for source_df, source_name in zip(dataframes, source_names):
        source_df.insert(0, 'Source_Name', source_name)

In [ ]:
add_source_name_column(lanka_news_sources, source_names)

In [ ]:
print(lanka_news_sources)

[         Source_Name                                           Articles
0    Ada Derena News  (2, 'CBSL decides not to renew permits of 15 m...
1    Ada Derena News  (2, 'X-Press Pearl disaster: SC calls for repo...
2    Ada Derena News  (2, 'Sri Lanka News | Breaking News in Sri Lan...
3    Ada Derena News  (2, 'Torrential rain, floods kill 22 across no...
4    Ada Derena News  (2, 'Driver arrested over bus accident in Mana...
..               ...                                                ...
775  Ada Derena News  (2, 'සුදුවෙන්න ගිය අයගේ ඇත්ත තත්ත්වය මෙන්න.. "...
776  Ada Derena News  (2, 'යළිත් වාහන ආනයනයට ඉඩ ලැබෙන්නේ මෙහෙමයි.. "...
777  Ada Derena News  (2, 'කුරුන්දි විහාරයේ ප්\u200dරශ්නය දුරදිග ගෙන...
778  Ada Derena News  (2, 'Belarus starts taking delivery of Russian...
779  Ada Derena News  (2, 'Former MP arrested for firing gun into ai...

[780 rows x 2 columns],                 Source_Name                                           Articles
0    Ada Derana Sports News  (6

In [ ]:
lanka_news_sources_df = pd.concat(lanka_news_sources)

In [ ]:
from ast import literal_eval as make_tuple

In [ ]:
article_list = []
for i in lanka_news_sources_df['Articles'].values.tolist() :
    article_list.append(make_tuple(i))

In [ ]:
corpus = pd.DataFrame(article_list , columns = ['news_source_id','Articles'])

In [ ]:
corpus

,news_source_id,Articles
0,2,CBSL decides not to renew permits of 15 money ...
1,2,X-Press Pearl disaster: SC calls for report on...
2,2,Sri Lanka News | Breaking News in Sri Lanka | ...
3,2,"Torrential rain, floods kill 22 across norther..."
4,2,Driver arrested over bus accident in Manampiti...
...,...,...
8300,10,Special commodity levy imposed on imported eg...
8301,10,“This happened with full knowledge of Presiden...
8302,10,New LG election dates to be announced on March...
8303,10,HRCSL to look into prevailing shortage of medi...


In [ ]:
corpus.to_csv('intermediate_file_1.csv', index = False)

In [ ]:
"""
You are given a collection of extracted news articles scraped from Sri Lankan news sources over the past 12 months.
Clean the dataset by removing null (or nearly empty),
non-English and possible duplicate articles and describe the resulting dataset in terms of the total number
of articles and the number of articles from each source.

Describe this cleaned corpus in terms of the total
number of articles and the number of articles from each source.

In preparation for building a classifier of
such news articles, address any potential imbalance in the dataset and describe the resulting final dataset
in terms of the total number of articles and the number of articles from each source. Pre-process the corpus
using the standard pipeline and describe the final dataset in terms the total number of tokens and the number
of unique tokens in the full corpus.
"""

'\nYou are given a collection of extracted news articles scraped from Sri Lankan news sources over the past 12 months.\nClean the dataset by removing null (or nearly empty),\nnon-English and possible duplicate articles and describe the resulting dataset in terms of the total number\nof articles and the number of articles from each source.\n\nDescribe this cleaned corpus in terms of the total\nnumber of articles and the number of articles from each source. \n\nIn preparation for building a classifier of\nsuch news articles, address any potential imbalance in the dataset and describe the resulting final dataset\nin terms of the total number of articles and the number of articles from each source. Pre-process the corpus\nusing the standard pipeline and describe the final dataset in terms the total number of tokens and the number\nof unique tokens in the full corpus.\n'

## Analyse the News articles

How to analyse the documents under each source ?

In text analytics before applying the clustering algorithms we do a data profile check on the articles we have. This involves understanding the characteristics and patterns of the data.   Initially we are curious to check whether there are null or nearly empty, non-English and possible duplicated articles. <br>

(1). Document length distribution --> Examine the distribution of document lengths to understand the variation in document sizes. <br>
(2). Domain Specific analyses --> Consider domain-specific analyses to gain insights into the context of the documents.
Identify key terms or topics that are relevant to the domain.<br>
(3). Data Exploration --> Check for any outliers or anomalies in the data.Explore basic statistics such as document length, word frequency, and vocabulary size.<br>
(4). Text preprocessing --> Tokenize the text to break it down into words or phrases.
Remove stop words (common words that don't carry much meaning) and punctuation.
Perform stemming or lemmatization to reduce words to their base form.<br>
(5). Feature Extraction --> Convert the text data into a numerical format suitable for clustering algorithms.
Use techniques like TF-IDF (Term Frequency-Inverse Document Frequency) to represent the importance of words in the documents.<br>
(6). Term Frequency Analysis--> Analyze the term frequency distribution to identify frequently occurring words.
Plot word clouds or histograms to visualize the most common terms.


## Data Cleaning

In [ ]:
corpus.groupby(['news_source_id']).count()

,Articles
news_source_id,
1,789
2,780
3,822
4,716
5,896
6,342
7,998
8,1005
9,982


### 1. Remove non english

In the data, observed some html embedded strings , non english content and time stamps. It was considered as the first step, because if we remove the empty articles or nearly empty articles, we will miss the articles where we already clean by this step. 

In [ ]:
import re
def is_english(text):
    english_pattern = re.compile(r'[^a-zA-Z\s]')
    return re.sub(english_pattern, '', text)


In [ ]:
def remove_html_markup(text):
    # Remove HTML markup
    text = re.sub('<.*?>', '', text)
    return text

In [ ]:
def show_non_english(text):
    english_pattern = re.compile(r'[a-zA-Z\s]')
    return re.sub(english_pattern, '', text)

In [ ]:
def remove_non_english_articles(df, col = ''):
    # Remove non-English article
    df[col] = df[col].apply(lambda x: remove_html_markup(x))
    df['non_english'] = df[col].apply(lambda x: show_non_english(x))
    df['english_context'] = df[col].apply(lambda x: is_english(x))
    
    return df

In [ ]:
corpus = remove_non_english_articles(corpus , 'Articles')

Here we detect the regex pattern [^a-zA-Z0-9.,!?;:()"'\s] which matches any character that is not an English alphabet, number, common English punctuation, or whitespace. The re.sub function is then used to replace those non-English characters with an empty string.

In [ ]:
corpus

,news_source_id,Articles,non_english,english_context
0,2,CBSL decides not to renew permits of 15 money ...,"1510,20238:351510,202303:27()15()2023.,-2022.,...",CBSL decides not to renew permits of money ch...
1,2,X-Press Pearl disaster: SC calls for report on...,"-:10,20238:36-:10,202303:18(10)-,,15.,,(10).-2...",XPress Pearl disaster SC calls for report on c...
2,2,Sri Lanka News | Breaking News in Sri Lanka | ...,"||-10,20238:3510(09),10...|10,202311:2410(09),...",Sri Lanka News Breaking News in Sri Lanka Ad...
3,2,"Torrential rain, floods kill 22 across norther...",",2210,20238:36,2210,202302:2822,,.,.22,,,,.,.,...",Torrential rain floods kill across northern I...
4,2,Driver arrested over bus accident in Manampiti...,"1010,20238:361010,202301:58(09).,,,.,.,,.,10,4...",Driver arrested over bus accident in Manampiti...
...,...,...,...,...
8300,10,Special commodity levy imposed on imported eg...,"|10,2023සිංහල|தமிழ்|252023|(27),.02().,02.,.,(...",Special commodity levy imposed on imported eg...
8301,10,“This happened with full knowledge of Presiden...,"“”:|10,2023සිංහල|தமிழ்|“”:252023|()().,,,.(23)...",This happened with full knowledge of President...
8302,10,New LG election dates to be announced on March...,"03|10,2023සිංහල|தமிழ்|03242023|202303,2023,.,0...",New LG election dates to be announced on March...
8303,10,HRCSL to look into prevailing shortage of medi...,"|10,2023සිංහල|தமிழ்|242023|(),.,.’,.,-()..,,.(...",HRCSL to look into prevailing shortage of medi...


### 2. Remove null records or nearly empty

To remove nearly empty records, I considered that there should be atleast with the noice data even after cleaning inappropriate content, we need to have atleast the word count 15. 

In [ ]:

def clear_null_records_or_nearly_empty(df , col = ''):
    # Remove null or nearly empty articles
    df.dropna(subset=[col], inplace=True)

    # Assuming we want to remove articles with less than 10 words (we can adjust the threshold)
    df['word_count'] = df[col].apply(lambda x: len(str(x).split()))
    df = df[df['word_count'] >= 15]


    # Display information after removing null or nearly empty articles
    print("\nDataset Information After Removing Null or Nearly Empty Articles:")
    print(df.info())
    return df

In [ ]:
corpus = clear_null_records_or_nearly_empty(corpus , 'english_context')


Dataset Information After Removing Null or Nearly Empty Articles:
<class 'pandas.core.frame.DataFrame'>
Index: 7549 entries, 0 to 8304
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   news_source_id   7549 non-null   int64 
 1   Articles         7549 non-null   object
 2   non_english      7549 non-null   object
 3   english_context  7549 non-null   object
 4   word_count       7549 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 353.9+ KB
None


In [ ]:
corpus

,news_source_id,Articles,non_english,english_context,word_count
0,2,CBSL decides not to renew permits of 15 money ...,"1510,20238:351510,202303:27()15()2023.,-2022.,...",CBSL decides not to renew permits of money ch...,575
1,2,X-Press Pearl disaster: SC calls for report on...,"-:10,20238:36-:10,202303:18(10)-,,15.,,(10).-2...",XPress Pearl disaster SC calls for report on c...,511
2,2,Sri Lanka News | Breaking News in Sri Lanka | ...,"||-10,20238:3510(09),10...|10,202311:2410(09),...",Sri Lanka News Breaking News in Sri Lanka Ad...,4048
3,2,"Torrential rain, floods kill 22 across norther...",",2210,20238:36,2210,202302:2822,,.,.22,,,,.,.,...",Torrential rain floods kill across northern I...,553
4,2,Driver arrested over bus accident in Manampiti...,"1010,20238:361010,202301:58(09).,,,.,.,,.,10,4...",Driver arrested over bus accident in Manampiti...,518
...,...,...,...,...,...
8300,10,Special commodity levy imposed on imported eg...,"|10,2023සිංහල|தமிழ்|252023|(27),.02().,02.,.,(...",Special commodity levy imposed on imported eg...,215
8301,10,“This happened with full knowledge of Presiden...,"“”:|10,2023සිංහල|தமிழ்|“”:252023|()().,,,.(23)...",This happened with full knowledge of President...,445
8302,10,New LG election dates to be announced on March...,"03|10,2023සිංහල|தமிழ்|03242023|202303,2023,.,0...",New LG election dates to be announced on March...,192
8303,10,HRCSL to look into prevailing shortage of medi...,"|10,2023සිංහල|தமிழ்|242023|(),.,.’,.,-()..,,.(...",HRCSL to look into prevailing shortage of medi...,187


In [ ]:
corpus.to_csv('nearlu_empty.csv')

In [ ]:
corpus.columns

Index(['news_source_id', 'Articles', 'non_english', 'english_context',
       'word_count'],
      dtype='object')

### 3. Remove duplicated using Natural Language Processing

Removing duplicates in a corpus using Natural Language Processing (NLP) involves comparing and identifying similar or identical text documents. The common approach is by using similarity measures and tokenization. We'll use the spaCy library for tokenization and the TfidfVectorizer from scikit-learn for creating document vectors.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

## Pre-process the corpus using the standard pipeline

Inorder to use cosine similarity score we need to prepare the set of unique tokens apply for each article. Therefore we need to tokenize the articles and convert it to vectors for easy execution. Therefore, data preprocessing will be done here itself. 

#### Tokenization

In [ ]:
from contractions import CONTRACTION_MAP
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer

In [ ]:
stopword_list = nltk.corpus.stopwords.words('english')
wnl = WordNetLemmatizer()

In [ ]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

def expand_contractions(text, contraction_mapping):

    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


# from pattern.en import tag (if using python 2.x)
from nltk.corpus import wordnet as wn


# Annotate text tokens with POS tags
def pos_tag_text(text):

    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None

    tokens = nltk.word_tokenize(text)
    tagged_text = nltk.pos_tag(tokens)

#    tagged_text = tag(text) # If using pattern.en in python 2.x
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
                         for word, pos_tag in
                         tagged_text]

    return tagged_lower_text

# Lemmatize text based on POS tags using Wordnet
def lemmatize_text(text):

    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                         else word
                         for word, pos_tag in pos_tagged_text]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text


def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


# Do all the pre-processing needed for the corpus
# Calls all the above functions to achieve this

def normalize_corpus(text):
    
    text = expand_contractions(text, CONTRACTION_MAP)
    text = lemmatize_text(text)
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    text = tokenize_text(text)

    return text


In [ ]:
corpus['preprocessed_article'] = corpus['english_context'].apply(lambda x : normalize_corpus(x))

C:\Users\hakim\AppData\Local\Temp\ipykernel_2200\2351256535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['preprocessed_article'] = corpus['english_context'].apply(lambda x : normalize_corpus(x))


In [ ]:
corpus.to_csv('preprocessed.csv', index = False)

In [ ]:
preprocessed_corpus = pd.read_csv('preprocessed.csv')

In [ ]:
preprocessed_corpus

,Unnamed: 0,news_source_id,Articles,non_english,english_context,word_count,preprocessed_article
0,0,2,CBSL decides not to renew permits of 15 money ...,"1510,20238:351510,202303:27()15()2023.,-2022.,...",CBSL decides not to renew permits of money ch...,575,"['cbsl', 'decide', 'renew', 'permit', 'money',..."
1,1,2,X-Press Pearl disaster: SC calls for report on...,"-:10,20238:36-:10,202303:18(10)-,,15.,,(10).-2...",XPress Pearl disaster SC calls for report on c...,511,"['xpress', 'pearl', 'disaster', 'sc', 'call', ..."
2,2,2,Sri Lanka News | Breaking News in Sri Lanka | ...,"||-10,20238:3510(09),10...|10,202311:2410(09),...",Sri Lanka News Breaking News in Sri Lanka Ad...,4048,"['sri', 'lanka', 'news', 'breaking', 'news', '..."
3,3,2,"Torrential rain, floods kill 22 across norther...",",2210,20238:36,2210,202302:2822,,.,.22,,,,.,.,...",Torrential rain floods kill across northern I...,553,"['torrential', 'rain', 'flood', 'kill', 'acros..."
4,4,2,Driver arrested over bus accident in Manampiti...,"1010,20238:361010,202301:58(09).,,,.,.,,.,10,4...",Driver arrested over bus accident in Manampiti...,518,"['driver', 'arrest', 'bus', 'accident', 'manam..."
...,...,...,...,...,...,...,...
7544,8300,10,Special commodity levy imposed on imported eg...,"|10,2023සිංහල|தமிழ்|252023|(27),.02().,02.,.,(...",Special commodity levy imposed on imported eg...,215,"['special', 'commodity', 'levy', 'impose', 'im..."
7545,8301,10,“This happened with full knowledge of Presiden...,"“”:|10,2023සිංහල|தமிழ்|“”:252023|()().,,,.(23)...",This happened with full knowledge of President...,445,"['happen', 'full', 'knowledge', 'president', '..."
7546,8302,10,New LG election dates to be announced on March...,"03|10,2023සිංහල|தமிழ்|03242023|202303,2023,.,0...",New LG election dates to be announced on March...,192,"['new', 'lg', 'election', 'date', 'announce', ..."
7547,8303,10,HRCSL to look into prevailing shortage of medi...,"|10,2023සිංහල|தமிழ்|242023|(),.,.’,.,-()..,,.(...",HRCSL to look into prevailing shortage of medi...,187,"['hrcsl', 'look', 'prevailing', 'shortage', 'm..."


### vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Create document vectors using TfidfVectorizer
vectorizer = TfidfVectorizer()
document_vectors = vectorizer.fit_transform(preprocessed_corpus['preprocessed_article'])

In [ ]:
print(document_vectors)

  (0, 25390)	0.028450965134936432
  (0, 49872)	0.02101600597108272
  (0, 68690)	0.02670228817975667
  (0, 58307)	0.03188554928242011
  (0, 3205)	0.03196675793906328
  (0, 52109)	0.03196675793906328
  (0, 73470)	0.03196675793906328
  (0, 7860)	0.03196675793906328
  (0, 73343)	0.03196675793906328
  (0, 45226)	0.03196675793906328
  (0, 6682)	0.03196675793906328
  (0, 6681)	0.03196675793906328
  (0, 24893)	0.05008449058167206
  (0, 58708)	0.06144283730582844
  (0, 5324)	0.04425766860807715
  (0, 14974)	0.06377109856484021
  (0, 53184)	0.048770657384059744
  (0, 24725)	0.05561642723586547
  (0, 41873)	0.06377109856484021
  (0, 17604)	0.05249684569453756
  (0, 646)	0.044278962954361824
  (0, 66536)	0.0449301075303929
  (0, 70998)	0.05404308986567626
  (0, 61855)	0.036752954381520625
  (0, 75716)	0.06377109856484021
  :	:
  (7548, 42217)	0.021600398410549768
  (7548, 38227)	0.03213302995950572
  (7548, 24452)	0.040552048881110914
  (7548, 37947)	0.4663425344425267
  (7548, 26606)	0.0244416722

### Cosine Similarity

cosine_similarity score says , when we pass two sentences or articles, how similar (returns percentage value)  are those articles are. Here when filtering out the duplicate documents,with considering noise data, to remove the duplicates atleast the score should be more than 80%. 

In [ ]:
# Calculate cosine similarity between document vectors
cosine_similarities = cosine_similarity(document_vectors, document_vectors)

In [ ]:
cosine_similarities.shape

(7549, 7549)

In [ ]:
# Threshold for considering documents as duplicates (adjust as needed)
duplicate_threshold = 0.8

In [ ]:
# Identify and remove duplicates
indices_to_remove = set()
for i in range(len(preprocessed_corpus)):

    if i not in indices_to_remove:
        similar_indices = set(index for index, score in enumerate(cosine_similarities[i]) if score > duplicate_threshold)

        if len(similar_indices) > 1 :
            indices_to_remove.update(similar_indices)


In [ ]:
# Drop duplicate indices
df_no_duplicates = preprocessed_corpus.drop(index=indices_to_remove)

# Display the resulting dataset without duplicates
print("Dataset without duplicates:")
print(df_no_duplicates.head())

Dataset without duplicates:
   Unnamed: 0  news_source_id  \
0           0               2   
1           1               2   
3           3               2   
5           5               2   
7          12               2   

                                            Articles  \
0  CBSL decides not to renew permits of 15 money ...   
1  X-Press Pearl disaster: SC calls for report on...   
3  Torrential rain, floods kill 22 across norther...   
5   India says it will continue to support Sri La...   
7  Pope Francis names 21 new cardinalsJuly 10, 20...   

                                         non_english  \
0  1510,20238:351510,202303:27()15()2023.,-2022.,...   
1  -:10,20238:36-:10,202303:18(10)-,,15.,,(10).-2...   
3  ,2210,20238:36,2210,202302:2822,,.,.22,,,,.,.,...   
5  -|සිංහල,10,2023’.---""'.,2023,’.--.“.-,”.().,1...   
7  2110,20238:362110,202312:1821,——..’,.30.,,,á,5...   

                                     english_context  word_count  \
0  CBSL decides not to renew pe

In [ ]:
df_no_duplicates.columns

Index(['Unnamed: 0', 'news_source_id', 'Articles', 'non_english',
       'english_context', 'word_count', 'preprocessed_article'],
      dtype='object')

In [ ]:
corpus_preprocessed = df_no_duplicates[['news_source_id' , 'preprocessed_article']].rename(columns={'preprocessed_article': 'Articles'})

In [ ]:
corpus_preprocessed

,news_source_id,Articles
0,2,"['cbsl', 'decide', 'renew', 'permit', 'money',..."
1,2,"['xpress', 'pearl', 'disaster', 'sc', 'call', ..."
3,2,"['torrential', 'rain', 'flood', 'kill', 'acros..."
5,2,"['india', 'say', 'continue', 'support', 'sri',..."
7,2,"['pope', 'francis', 'names', 'new', 'cardinals..."
...,...,...
7544,10,"['special', 'commodity', 'levy', 'impose', 'im..."
7545,10,"['happen', 'full', 'knowledge', 'president', '..."
7546,10,"['new', 'lg', 'election', 'date', 'announce', ..."
7547,10,"['hrcsl', 'look', 'prevailing', 'shortage', 'm..."


In [ ]:
corpus_preprocessed.to_csv('final_preprocessed.csv', index = False)

In [ ]:
corpus_preprocessed_ = pd.read_csv('final_preprocessed.csv')

Describe the resulting dataset in terms of the total number of articles and the number of articles from each source.

In [ ]:
corpus_preprocessed_.count()

news_source_id    4585
Articles          4585
dtype: int64

There are 4585 articles in total in the corpus after removing the duplicates. To observe or detect imbalance data, we will check how the distribution of count of articles from each source,

In [ ]:
agg_corpus = corpus_preprocessed_.groupby(['news_source_id']).count()
agg_corpus

,Articles
news_source_id,
1,531
2,377
3,464
4,170
5,655
6,24
7,535
8,901
9,87


By identifying the outliers we consider the clusters which have article count out of the range of Q1 and Q2. Then to the resampling size, by calculating the trimmed mean.  

In [ ]:
agg_corpus.Articles.mean()

458.5

In [ ]:
from scipy import stats
#calculate 10% trimmed mean
stats.trim_mean(agg_corpus, 0.1)

array([457.5])

In [ ]:
trimmed_mean  = stats.trim_mean(agg_corpus, 0.1)
trimmed_mean

array([457.5])

In [ ]:
# First quartile (Q1)
Q1 = np.percentile(agg_corpus, 25, interpolation = 'midpoint')
print('First quatile : ' , Q1)
# Third quartile (Q3)
Q3 = np.percentile(agg_corpus, 75, interpolation = 'midpoint')
print('Third quatile : ' , Q3)
# Interquaritle range (IQR)
IQR = Q3 - Q1

print('Interquartile',IQR)

First quatile :  273.5
Third quatile :  595.0
Interquartile 321.5


C:\Users\hakim\AppData\Local\Temp\ipykernel_2200\1354273021.py:2: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  Q1 = np.percentile(agg_corpus, 25, interpolation = 'midpoint')
C:\Users\hakim\AppData\Local\Temp\ipykernel_2200\1354273021.py:5: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  Q3 = np.percentile(agg_corpus, 75, interpolation = 'midpoint')


In preparation for building a classifier of
such news articles, address any potential imbalance in the dataset.

Tried to import imblearn to get SMOTE , but it didn't work.

In [ ]:
from sklearn.utils import resample

### Identify undersampled and oversampled clusters

In [ ]:
agg_corpus['diff'] = agg_corpus['Articles'].apply(lambda x : trimmed_mean - x)
upper_boundary =Q3
lower_boundary =Q1
print('upper_boundary : ' ,upper_boundary , ' lower_boundary:' , lower_boundary)
agg_corpus['cluster_status'] = (agg_corpus['Articles'].apply(
                                                        lambda x : "Undersampled"
                                                        if x < lower_boundary
                                                        else (
                                                            "Sampled_enough"
                                                            if ((lower_boundary <= x) and (x <=upper_boundary  ))
                                                            else "Oversampled") )
                                                                                   )

upper_boundary :  595.0  lower_boundary: 273.5


In [ ]:
agg_corpus

,Articles,diff,cluster_status
news_source_id,,,
1,531,[-73.5],Sampled_enough
2,377,[80.5],Sampled_enough
3,464,[-6.5],Sampled_enough
4,170,[287.5],Undersampled
5,655,[-197.5],Oversampled
6,24,[433.5],Undersampled
7,535,[-77.5],Sampled_enough
8,901,[-443.5],Oversampled
9,87,[370.5],Undersampled


In [ ]:
# Define a condition
condition = (agg_corpus['cluster_status']== 'Undersampled')

# Use boolean indexing to get the DataFrame with rows that satisfy the condition
filtered_df = agg_corpus[condition]

# Get the index of the rows that satisfy the condition
undersampled_clusters = filtered_df.index

In [ ]:
undersampled_clusters

Index([4, 6, 9], dtype='int64', name='news_source_id')

In [ ]:
# Define a condition
condition2 = (agg_corpus['cluster_status']== 'Oversampled')

# Use boolean indexing to get the DataFrame with rows that satisfy the condition
filtered_df2 = agg_corpus[condition2]

# Get the index of the rows that satisfy the condition
oversampled_clusters = filtered_df2.index

In [ ]:
oversampled_clusters

Index([5, 8, 10], dtype='int64', name='news_source_id')

## Handling undersampled data clusters

In [ ]:
def resampling_undersample_clusters(df_minor):

    # Upsampling minority class
    df_minor_sample = resample(df_minor,

                               # Upsample with replacement
                               replace=True,

                               # Number to match majority class
                               n_samples= int(trimmed_mean),
                               random_state=42)
    return df_minor_sample

In [ ]:
for i in range(0, len(undersampled_clusters)):
    df_minor = corpus_preprocessed_[corpus_preprocessed_.news_source_id == undersampled_clusters[i] ]
    df_minor_sample = resampling_undersample_clusters(df_minor)

    corpus_preprocessed_ = corpus_preprocessed_[corpus_preprocessed_.news_source_id != undersampled_clusters[i]]
    corpus_preprocessed_ = pd.concat([corpus_preprocessed_ , df_minor_sample])

## Handling oversampled data clusters

In [ ]:
def resampling_oversample_clusters(df_major):

    # Upsampling minority class
    df_major_sample = resample(df_major,

                               # Oversample with replacement
                               replace=True,

                               # Number to match majority class
                               n_samples=int(trimmed_mean),
                               random_state=42)
    return df_major_sample

In [ ]:
for i in range(0, len(oversampled_clusters)):
    df_major = corpus_preprocessed_[corpus_preprocessed_.news_source_id == oversampled_clusters[i]]
    df_major_sample = resampling_oversample_clusters(df_major)

    corpus_preprocessed_ = corpus_preprocessed_[corpus_preprocessed_.news_source_id != oversampled_clusters[i]]
    corpus_preprocessed_ = pd.concat([corpus_preprocessed_ , df_major_sample])

Describe the resulting final dataset
in terms of the total number of articles and the number of articles from each source.

In [ ]:
corpus_preprocessed_.groupby(['news_source_id']).count()

,Articles
news_source_id,
1,531
2,377
3,464
4,457
5,457
6,457
7,535
8,457
9,457


In [ ]:
corpus_preprocessed_

,news_source_id,Articles
0,2,"['cbsl', 'decide', 'renew', 'permit', 'money',..."
1,2,"['xpress', 'pearl', 'disaster', 'sc', 'call', ..."
2,2,"['torrential', 'rain', 'flood', 'kill', 'acros..."
3,2,"['india', 'say', 'continue', 'support', 'sri',..."
4,2,"['pope', 'francis', 'names', 'new', 'cardinals..."
...,...,...
4249,10,"['government', 'race', 'time', 'finalise', 'de..."
4568,10,"['mr', 'back', 'pm', 'morningth', 'july', 'e',..."
3779,10,"['foreigner', 'sentence', 'death', 'drug', 'tr..."
4428,10,"['ntuc', 'union', 'give', 'week', 'ultimatum',..."


----------------------------------------

In [ ]:
corpus_normalized = corpus_preprocessed_

## PART B

In [ ]:
"""
Use one sparse and one dense vector representation for extracting features for training a classifier for this dataset.

State and interpret the dimensions of the sparse matrix of the resulting dataset and justify the dimensions of the dense
matrix of the resulting dataset respectively.
"""

'\nUse one sparse and one dense vector representation for extracting features for training a classifier for this dataset.\n\nState and interpret the dimensions of the sparse matrix of the resulting dataset and justify the dimensions of the dense\nmatrix of the resulting dataset respectively.\n'

Use one sparse and one dense vector representation for extracting features for training a classifier for this dataset.

## Extract features using sparse data representation

Term Frequency -->  means we summarize how often a given word appears within a document.<br>
Inverse Document Frequency --> means figuring out the words that appear a lot across documents.<br>

TF-IDF build a vocabulary of words which it has learned from the corpus data and it will assign a unique integer number to each of these words. Their will be maximum of 5000 unique words/features as we have set parameter max_features=5000.

In [ ]:
corpus_normalized

,news_source_id,Articles
0,2,"['cbsl', 'decide', 'renew', 'permit', 'money',..."
1,2,"['xpress', 'pearl', 'disaster', 'sc', 'call', ..."
2,2,"['torrential', 'rain', 'flood', 'kill', 'acros..."
3,2,"['india', 'say', 'continue', 'support', 'sri',..."
4,2,"['pope', 'francis', 'names', 'new', 'cardinals..."
...,...,...
4249,10,"['government', 'race', 'time', 'finalise', 'de..."
4568,10,"['mr', 'back', 'pm', 'morningth', 'july', 'e',..."
3779,10,"['foreigner', 'sentence', 'death', 'drug', 'tr..."
4428,10,"['ntuc', 'union', 'give', 'week', 'ultimatum',..."


### Feature : Weight of each term

Term Frequency-Inverse Document Frequency (TF-IDF): Weighing the importance of words in a document relative to their frequency in the entire corpus.

Initially it was number_of_tokens as features were (4649, 55857). But due to limited computation power, had to set a boundary for the features

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()


# Fit and transform the tokenized_content to create TF-IDF representation
sparse_vector = tfidf_vectorizer.fit_transform(corpus_normalized['Articles'])


In [ ]:
print(sparse_vector)

  (0, 18161)	0.025806700826348773
  (0, 35432)	0.02053093713350331
  (0, 49119)	0.024402895484684144
  (0, 41638)	0.03061448967695374
  (0, 2288)	0.03247754028530527
  (0, 37043)	0.03247754028530527
  (0, 52521)	0.03247754028530527
  (0, 5592)	0.03247754028530527
  (0, 52436)	0.03247754028530527
  (0, 32343)	0.03247754028530527
  (0, 4717)	0.03247754028530527
  (0, 4716)	0.03247754028530527
  (0, 17817)	0.04746125473254978
  (0, 41902)	0.062389491129382774
  (0, 3747)	0.04613680351664929
  (0, 10624)	0.06495508057061054
  (0, 37826)	0.048440255984750644
  (0, 17695)	0.054279845444213236
  (0, 30116)	0.06495508057061054
  (0, 12467)	0.04966642132791416
  (0, 470)	0.04190355205032465
  (0, 47533)	0.04761445123005975
  (0, 50764)	0.053391379629586
  (0, 44236)	0.03692217602173713
  (0, 54161)	0.06495508057061054
  :	:
  (4648, 31142)	0.13758355066292438
  (4648, 25253)	0.013987352290760178
  (4648, 50764)	0.10832023865656018
  (4648, 44236)	0.018726894426314252
  (4648, 32277)	0.023207396



tfidf_matrix is a sparse matrix where each row corresponds to a document, and each column corresponds to a unique word in the corpus. The entries in the matrix represent the TF-IDF score of each word in each document.

Number of article , Uniuqe tokens

In [ ]:
print(sparse_vector.shape)

(4649, 55857)


In [ ]:
# Get the feature names (unique tokens)
feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()
print("feature_names_tfidf")
print(feature_names_tfidf)

feature_names_tfidf
['aa' 'aaa' 'aad' ... 'zuraish' 'zurich' 'zwerner']


In [ ]:
len(feature_names_tfidf)

55857

### Sparse vector representation - Test/Train data

In [ ]:
#spliting the dataset
sparse_train_X, sparse_test_X, sparse_train_Y, sparse_test_Y = model_selection.train_test_split(sparse_vector,corpus_normalized['news_source_id'],test_size=0.3)

State and interpret the dimensions of the sparse matrix of the resulting dataset

In [ ]:
print(sparse_train_X.shape)

(3254, 55857)


In [ ]:
print(sparse_test_X.shape)

(1395, 55857)


In [ ]:
print(sparse_train_Y.shape)

(3254,)


In [ ]:
print(sparse_test_Y.shape)

(1395,)


In [ ]:
print(sparse_train_X)

  (0, 620)	0.07196887367812128
  (0, 40828)	0.07196887367812128
  (0, 43019)	0.06863447963758372
  (0, 20575)	0.07196887367812128
  (0, 28129)	0.06626868941600733
  (0, 20576)	0.07196887367812128
  (0, 32336)	0.07196887367812128
  (0, 44601)	0.07196887367812128
  (0, 52491)	0.04969887739123259
  (0, 30517)	0.04969887739123259
  (0, 1219)	0.04969887739123259
  (0, 1259)	0.04969887739123259
  (0, 12857)	0.04969887739123259
  (0, 1222)	0.04942922610034495
  (0, 1226)	0.04969887739123259
  (0, 24450)	0.04969887739123259
  (0, 1217)	0.04969887739123259
  (0, 22814)	0.03175246377450839
  (0, 16501)	0.05026624907112643
  (0, 35035)	0.04969887739123259
  (0, 11259)	0.04969887739123259
  (0, 35227)	0.04969887739123259
  (0, 8599)	0.04969887739123259
  (0, 32462)	0.04969887739123259
  (0, 48826)	0.04969887739123259
  :	:
  (3253, 34746)	0.021194325292568858
  (3253, 23996)	0.015291433417543009
  (3253, 36482)	0.013838354000376194
  (3253, 27779)	0.026451831115369074
  (3253, 8894)	0.028159371645

## Extract features using dense data representation

Here due to computation limitations, we have limited the number of features. By setting vector_size=1000, it will fetch best 1000 features by considering the vector value.  

In [ ]:
import gensim
import nltk

TOKENIZED_CORPUS = [nltk.word_tokenize(sentence)
                    for sentence in corpus_normalized['Articles']]


In [ ]:

# Model parameters for the NN-based word2vec 'word embeddings':
# size - dimension of the word vectors (tens to thousands)
# window - window size to conside the context of a word
# min_count - the minimum frequency of a word in the whole corpus to be included in vocabulary
# sample - used to downsample the effects of the occurence of frequent words
model = gensim.models.Word2Vec(TOKENIZED_CORPUS,
                               vector_size=1000,
                               window=8,
                               min_count=1,
                               workers = 4 )

In [ ]:

# Function to convert tokenized content into dense vectors
def get_doc_embedding(doc_tokens):
    embeddings = [model.wv[token] for token in doc_tokens if token in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    return np.zeros(model.vector_size)  # Return zeros for empty documents
# Apply the function to the 'tokenized_content' column and store the dense vectors in a new column 'dense_vectors'
corpus_normalized['dense_vectors'] = corpus_normalized['Articles'].apply(get_doc_embedding)

In [ ]:
corpus_normalized

,news_source_id,Articles,dense_vectors
0,2,"['cbsl', 'decide', 'renew', 'permit', 'money',...","[0.027175818, 0.47011563, -0.10083615, -0.1088..."
1,2,"['xpress', 'pearl', 'disaster', 'sc', 'call', ...","[0.0058154617, 0.48902646, -0.08228876, -0.095..."
2,2,"['torrential', 'rain', 'flood', 'kill', 'acros...","[0.017496474, 0.48102412, -0.09090834, -0.1054..."
3,2,"['india', 'say', 'continue', 'support', 'sri',...","[0.018921798, 0.48401517, -0.08358823, -0.1124..."
4,2,"['pope', 'francis', 'names', 'new', 'cardinals...","[0.025925383, 0.46417668, -0.09185875, -0.1063..."
...,...,...,...
4249,10,"['government', 'race', 'time', 'finalise', 'de...","[0.027893167, 0.475432, -0.105379924, -0.11472..."
4568,10,"['mr', 'back', 'pm', 'morningth', 'july', 'e',...","[0.017281014, 0.4736882, -0.09014147, -0.10513..."
3779,10,"['foreigner', 'sentence', 'death', 'drug', 'tr...","[0.01417211, 0.4507376, -0.078855336, -0.08280..."
4428,10,"['ntuc', 'union', 'give', 'week', 'ultimatum',...","[0.024382606, 0.48901093, -0.07192123, -0.1036..."


In [ ]:
corpus_normalized.to_csv('dense_corpus.csv', index= False)

### Dense vector representation - Test/Train data

In [ ]:
dense_corpus = np.vstack(corpus_normalized['dense_vectors'].to_numpy())

In [ ]:
#spliting the dataset
dense_train_X, dense_test_X, dense_train_Y, dense_test_Y = model_selection.train_test_split(dense_corpus,corpus_normalized['news_source_id'],test_size=0.3)

In [ ]:
dense_train_X.shape, dense_test_X.shape, dense_train_Y.shape, dense_test_Y.shape

((3254, 1000), (1395, 1000), (3254,), (1395,))

In [ ]:
dense_test_X

array([[ 0.01971179,  0.4839017 , -0.087449  , ...,  0.05809121,
        -0.10273505,  0.3569534 ],
       [ 0.00317502,  0.45791522, -0.08397418, ...,  0.07238246,
        -0.09684703,  0.30982164],
       [ 0.02487194,  0.46773273, -0.09541527, ...,  0.0614766 ,
        -0.11313436,  0.3460021 ],
       ...,
       [ 0.02386056,  0.46107197, -0.0617226 , ...,  0.06179084,
        -0.10115226,  0.32425943],
       [ 0.00849587,  0.46239763, -0.08683304, ...,  0.06600481,
        -0.09479102,  0.32984492],
       [ 0.02263385,  0.465001  , -0.06930453, ...,  0.05425067,
        -0.09862027,  0.33652294]], dtype=float32)

## PART C

In [ ]:
"""
Train classifiers with the two (02) representations above using an explainable baseline algorithm,
a more complex non-deep learning algorithm and a non-deep learning ensemble algorithm.

Compare and contrast
the performance of each of the classifiers and interpreting the results.

Cluster Validation:
If you have labeled data, perform cluster validation using metrics like silhouette score or Davies–Bouldin index
to assess the quality of the clusters.
"""

Train classifiers with the two (02) representations above using an explainable baseline algorithm,
a more complex non-deep learning algorithm and a non-deep learning ensemble algorithm.


(1). Explaninable baseline algorithm --> Logistic Regression <br>
(2). Complex non-deep learning algorithm --> Random Forest Classiier <br>
(3). Non-deep learning ensemble algorithm --> Support Vector Machine <br>

### Sparse Data Representaion

Explaining the advantages in sparse data representation

@ sparse data representation, it saves memory compared to dense representation and we would only store non-zero values.<br>
@ algorithms that take the advantage of sparsity like in linear regression algorithms, it reduce the complexity in sparse matrix operations.<br>
@ speed computation, because mathematical operations involving zero values can be avoided.<br>
@ improves model generalization by focusing on the most informative features, reducing the risk of overfitting

Explaining the disadvantages in sparse data representation

@  Will miss the relationships between features and their ordering.<br>
@ sparse can become imbalanced, it can only effect for the algorithms that are sensitive for class distribution. <br>
@ not suitable for small and moderate datasets. <br>

(1). Explaninable baseline algorithm --> Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# Initialize classifiers
logreg_clf = LogisticRegression()
# Training the classifiers
print("Training for LR")
logreg_clf.fit(sparse_train_X, sparse_train_Y)
# Making predictions
print("Predicting for LR")
y_pred_logreg = logreg_clf.predict(sparse_test_X)
# Evaluating the classifiers

Training for LR
Predicting for LR


In [ ]:
accuracy_logreg = accuracy_score(sparse_test_Y, y_pred_logreg)
print("Logistic Regression Accuracy:", accuracy_logreg)
print("Logistic Regression Classification Report:")
print(classification_report(sparse_test_Y, y_pred_logreg))


Logistic Regression Accuracy: 0.996415770609319
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       150
           2       1.00      0.98      0.99       103
           3       0.99      1.00      0.99       138
           4       1.00      0.98      0.99       132
           5       1.00      1.00      1.00       149
           6       0.99      1.00      1.00       158
           7       1.00      1.00      1.00       148
           8       0.99      1.00      1.00       132
           9       1.00      1.00      1.00       148
          10       0.99      0.99      0.99       137

    accuracy                           1.00      1395
   macro avg       1.00      1.00      1.00      1395
weighted avg       1.00      1.00      1.00      1395



(2). Complex non-deep learning algorithm --> Random Forest Classiier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Initialize classifiers
rf_clf = RandomForestClassifier()

In [ ]:
print("Training for RFC")
rf_clf.fit(sparse_train_X, sparse_train_Y)
# Making predictions
print("Predicting for RFC")
y_pred_rf = rf_clf.predict(sparse_test_X)

Training for RFC
Predicting for RFC


In [ ]:
# Evaluating the classifiers
accuracy_rf = accuracy_score(sparse_test_Y, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)
print("Random Forest Classification Report:")
print(classification_report(sparse_test_Y, y_pred_rf))

Random Forest Accuracy: 0.9971326164874552
Random Forest Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       150
           2       1.00      0.98      0.99       103
           3       0.98      1.00      0.99       138
           4       1.00      0.99      1.00       132
           5       1.00      1.00      1.00       149
           6       0.99      1.00      1.00       158
           7       1.00      1.00      1.00       148
           8       1.00      1.00      1.00       132
           9       1.00      1.00      1.00       148
          10       1.00      0.99      1.00       137

    accuracy                           1.00      1395
   macro avg       1.00      1.00      1.00      1395
weighted avg       1.00      1.00      1.00      1395



(3). Non-deep learning ensemble algorithm --> Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
# Initialize classifiers
svm_clf = SVC()
# Training the classifiers
print("Training for SVC")
svm_clf.fit(sparse_train_X, sparse_train_Y)
# Making predictions
print("Predicting for SVC")
y_pred_svm = svm_clf.predict(sparse_test_X)


Training for SVC
Predicting for SVC


In [ ]:
# Evaluating the classifiers
accuracy_svm = accuracy_score(sparse_test_Y, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)
print("SVM Classification Report:")
print(classification_report(sparse_test_Y, y_pred_svm))

SVM Accuracy: 0.996415770609319
SVM Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.99      1.00       150
           2       1.00      0.98      0.99       103
           3       0.99      1.00      0.99       138
           4       1.00      0.99      1.00       132
           5       1.00      1.00      1.00       149
           6       1.00      1.00      1.00       158
           7       1.00      1.00      1.00       148
           8       0.98      1.00      0.99       132
           9       1.00      1.00      1.00       148
          10       1.00      0.99      1.00       137

    accuracy                           1.00      1395
   macro avg       1.00      1.00      1.00      1395
weighted avg       1.00      1.00      1.00      1395



### Dense Data Representation

Explaining the advantages in sparse data representation

@ good for clustering algorithms that rely on distance metrics<br>
@ easy to implement and understand <br>
@ easy to debug <br>
@ suitable for small datasets

Explaining the disadvantages in sparse data representation

@ sensitive for the outlier data. <br>
@ because of high dimensionality, it's difficult to interpret or find meaningful patterns. <br>
@ it will take more computation time , therefore not suitable for real time applications or near real time applications.

(1). Explaninable baseline algorithm --> Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# Initialize the classifiers
clf_logreg = LogisticRegression(max_iter=1000, random_state=42)
# Train the classifiers
clf_logreg.fit(dense_train_X, dense_train_Y)
# Make predictions on the test set
y_pred_logreg = clf_logreg.predict(dense_test_X)


C:\Users\hakim\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Evaluate the classifiers
accuracy_logreg = accuracy_score(dense_test_Y, y_pred_logreg)
print("Accuracy (Logistic Regression):", accuracy_logreg)
print("\nClassification Report (Logistic Regression):\n", classification_report(dense_test_Y, y_pred_logreg))

Accuracy (Logistic Regression): 0.7985663082437277

Classification Report (Logistic Regression):
               precision    recall  f1-score   support

           1       0.69      0.95      0.80       153
           2       0.75      0.79      0.77       127
           3       0.83      0.70      0.76       132
           4       0.86      0.90      0.88       156
           5       0.89      0.86      0.88       144
           6       0.92      0.75      0.83       134
           7       0.89      0.94      0.91       142
           8       0.84      0.78      0.81       139
           9       0.73      0.89      0.80       120
          10       0.59      0.43      0.50       148

    accuracy                           0.80      1395
   macro avg       0.80      0.80      0.79      1395
weighted avg       0.80      0.80      0.79      1395



(2). Complex non-deep learning algorithm --> Random Forest Classiier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Initialize classifiers
rf_clf = RandomForestClassifier()

In [ ]:
print("Training for RFC")
rf_clf.fit(dense_train_X, dense_train_Y)
# Making predictions
print("Predicting for RFC")
y_pred_rf = rf_clf.predict(dense_test_X)

Training for RFC
Predicting for RFC


In [ ]:
# Evaluating the classifiers
accuracy_rf = accuracy_score(dense_test_Y, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)
print("Random Forest Classification Report:")
print(classification_report(dense_test_Y, y_pred_rf))

Random Forest Accuracy: 0.9154121863799283
Random Forest Classification Report:
              precision    recall  f1-score   support

           1       0.94      0.95      0.95       153
           2       0.83      0.86      0.84       127
           3       0.89      0.89      0.89       132
           4       0.94      0.97      0.96       156
           5       0.95      0.96      0.95       144
           6       0.99      1.00      0.99       134
           7       0.99      0.93      0.96       142
           8       0.93      0.90      0.91       139
           9       0.96      1.00      0.98       120
          10       0.74      0.70      0.72       148

    accuracy                           0.92      1395
   macro avg       0.91      0.92      0.92      1395
weighted avg       0.91      0.92      0.91      1395



(3). Non-deep learning ensemble algorithm --> Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
# Initialize classifiers
svm_clf = SVC()
# Training the classifiers
print("Training for SVC")
svm_clf.fit(dense_train_X, dense_train_Y)
# Making predictions
print("Predicting for SVC")
y_pred_svm = svm_clf.predict(dense_test_X)


Training for SVC
Predicting for SVC


In [ ]:
# Evaluating the classifiers
accuracy_svm = accuracy_score(dense_test_Y, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)
print("SVM Classification Report:")
print(classification_report(dense_test_Y, y_pred_svm))

SVM Accuracy: 0.36200716845878134
SVM Classification Report:
              precision    recall  f1-score   support

           1       0.18      0.99      0.31       153
           2       0.00      0.00      0.00       127
           3       0.86      0.29      0.43       132
           4       0.42      0.15      0.22       156
           5       0.67      0.21      0.32       144
           6       0.94      0.65      0.77       134
           7       0.46      0.87      0.60       142
           8       0.00      0.00      0.00       139
           9       0.76      0.31      0.44       120
          10       0.79      0.10      0.18       148

    accuracy                           0.36      1395
   macro avg       0.51      0.36      0.33      1395
weighted avg       0.50      0.36      0.32      1395



C:\Users\hakim\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hakim\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hakim\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Comparison


model | sparse | dense 
| ----------- | ----------- | ----------- |
Logistic Regrassion | 0.99 | 0.79
Random forest | 0.99 | 0.915
SVM | 0.99 | 0.36

 
 Compared to spare vector representation , reason for dense vector representation to have less accuracy is because of having more outlier articles based on the len of articles and because of high dimensionality, it's difficult to interpret or find meaningful patterns. With padding the sequences also cause the model to learn more zeros than the article content. 

# PART D

In [ ]:
"""
(d) Train also three (03) deep learning classifiers with distinct architectures
using two (02) embedding techniques and one (01) contextual embedding technique,
justifying the architectures you employ. Compare the performance of each of the models and interpret the results.
"""

'\n(d) Train also three (03) deep learning classifiers with distinct architectures \nusing two (02) embedding techniques and one (01) contextual embedding technique,\njustifying the architectures you employ. Compare the performance of each of the models and interpret the results.\n'

In [ ]:
def normalize_corpus_2(text):

    text = expand_contractions(text, CONTRACTION_MAP)
    text = lemmatize_text(text)
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    return text

In [ ]:
corpus['cleaned_article'] = corpus['english_context'].apply(lambda x : normalize_corpus_2(x))

C:\Users\hakim\AppData\Local\Temp\ipykernel_2200\3979527358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['cleaned_article'] = corpus['english_context'].apply(lambda x : normalize_corpus_2(x))


In [ ]:
corpus

,news_source_id,Articles,non_english,english_context,word_count,preprocessed_article,cleaned_article
0,2,CBSL decides not to renew permits of 15 money ...,"1510,20238:351510,202303:27()15()2023.,-2022.,...",CBSL decides not to renew permits of money ch...,575,"[cbsl, decide, renew, permit, money, changersj...",cbsl decide renew permit money changersjuly pm...
1,2,X-Press Pearl disaster: SC calls for report on...,"-:10,20238:36-:10,202303:18(10)-,,15.,,(10).-2...",XPress Pearl disaster SC calls for report on c...,511,"[xpress, pearl, disaster, sc, call, report, co...",xpress pearl disaster sc call report compensat...
2,2,Sri Lanka News | Breaking News in Sri Lanka | ...,"||-10,20238:3510(09),10...|10,202311:2410(09),...",Sri Lanka News Breaking News in Sri Lanka Ad...,4048,"[sri, lanka, news, breaking, news, sri, lanka,...",sri lanka news breaking news sri lanka adadera...
3,2,"Torrential rain, floods kill 22 across norther...",",2210,20238:36,2210,202302:2822,,.,.22,,,,.,.,...",Torrential rain floods kill across northern I...,553,"[torrential, rain, flood, kill, across, northe...",torrential rain flood kill across northern ind...
4,2,Driver arrested over bus accident in Manampiti...,"1010,20238:361010,202301:58(09).,,,.,.,,.,10,4...",Driver arrested over bus accident in Manampiti...,518,"[driver, arrest, bus, accident, manampitiya, l...",driver arrest bus accident manampitiya leave d...
...,...,...,...,...,...,...,...
8300,10,Special commodity levy imposed on imported eg...,"|10,2023සිංහල|தமிழ்|252023|(27),.02().,02.,.,(...",Special commodity levy imposed on imported eg...,215,"[special, commodity, levy, impose, import, egg...",special commodity levy impose import egg revis...
8301,10,“This happened with full knowledge of Presiden...,"“”:|10,2023සිංහල|தமிழ்|“”:252023|()().,,,.(23)...",This happened with full knowledge of President...,445,"[happen, full, knowledge, president, harsha, m...",happen full knowledge president harsha morning...
8302,10,New LG election dates to be announced on March...,"03|10,2023සිංහල|தமிழ்|03242023|202303,2023,.,0...",New LG election dates to be announced on March...,192,"[new, lg, election, date, announce, march, mor...",new lg election date announce march morningth ...
8303,10,HRCSL to look into prevailing shortage of medi...,"|10,2023සිංහල|தமிழ்|242023|(),.,.’,.,-()..,,.(...",HRCSL to look into prevailing shortage of medi...,187,"[hrcsl, look, prevailing, shortage, medicine, ...",hrcsl look prevailing shortage medicine mornin...


In [5]:
import pandas as pd

In [4]:
corpus = pd.read_csv('deep_learning_dataset_cleaned.csv')

#### 1. Transformer with GloVe Embeddings:
Architecture:

Input Layer: Tokenized sequences.<br>
Embedding Layer: GloVe Embeddings for capturing global semantic relationships.<br>
Transformer Encoder Layers: Self-attention mechanism for capturing long-range dependencies efficiently.<br>
Global Average Pooling: Aggregates information from the entire sequence.<br>
Dense Layers with Layer Normalization: Fully connected layers with layer normalization for better stability during training.<br>
Output Layer: Final classification layer.<br>

Justification:

GloVe Embeddings: Effective for capturing global semantic relationships.<br>
Transformer: Handles long-range dependencies efficiently, suitable for tasks where the context can be crucial regardless of the distance between words.<br>
These architectures are just starting points, and you may need to fine-tune them based on the specific requirements of your task and dataset. Experimenting with different architectures, hyperparameters, and embeddings is crucial to finding the most effective model for your application.

In [100]:


from google.colab import drive
drive.mount("/content/gdrive")

corpus = pd.read_csv('/content/gdrive/My Drive/text_analytics/deep_learning_dataset_cleaned.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [101]:
corpus

,news_source_id,Articles,non_english,english_context,word_count,preprocessed_article,cleaned_article
0,2,CBSL decides not to renew permits of 15 money ...,"1510,20238:351510,202303:27()15()2023.,-2022.,...",CBSL decides not to renew permits of money ch...,575,"['cbsl', 'decide', 'renew', 'permit', 'money',...",cbsl decide renew permit money changersjuly pm...
1,2,X-Press Pearl disaster: SC calls for report on...,"-:10,20238:36-:10,202303:18(10)-,,15.,,(10).-2...",XPress Pearl disaster SC calls for report on c...,511,"['xpress', 'pearl', 'disaster', 'sc', 'call', ...",xpress pearl disaster sc call report compensat...
2,2,Sri Lanka News | Breaking News in Sri Lanka | ...,"||-10,20238:3510(09),10...|10,202311:2410(09),...",Sri Lanka News Breaking News in Sri Lanka Ad...,4048,"['sri', 'lanka', 'news', 'breaking', 'news', '...",sri lanka news breaking news sri lanka adadera...
3,2,"Torrential rain, floods kill 22 across norther...",",2210,20238:36,2210,202302:2822,,.,.22,,,,.,.,...",Torrential rain floods kill across northern I...,553,"['torrential', 'rain', 'flood', 'kill', 'acros...",torrential rain flood kill across northern ind...
4,2,Driver arrested over bus accident in Manampiti...,"1010,20238:361010,202301:58(09).,,,.,.,,.,10,4...",Driver arrested over bus accident in Manampiti...,518,"['driver', 'arrest', 'bus', 'accident', 'manam...",driver arrest bus accident manampitiya leave d...
...,...,...,...,...,...,...,...
7544,10,Special commodity levy imposed on imported eg...,"|10,2023සිංහල|தமிழ்|252023|(27),.02().,02.,.,(...",Special commodity levy imposed on imported eg...,215,"['special', 'commodity', 'levy', 'impose', 'im...",special commodity levy impose import egg revis...
7545,10,“This happened with full knowledge of Presiden...,"“”:|10,2023සිංහල|தமிழ்|“”:252023|()().,,,.(23)...",This happened with full knowledge of President...,445,"['happen', 'full', 'knowledge', 'president', '...",happen full knowledge president harsha morning...
7546,10,New LG election dates to be announced on March...,"03|10,2023සිංහල|தமிழ்|03242023|202303,2023,.,0...",New LG election dates to be announced on March...,192,"['new', 'lg', 'election', 'date', 'announce', ...",new lg election date announce march morningth ...
7547,10,HRCSL to look into prevailing shortage of medi...,"|10,2023සිංහල|தமிழ்|242023|(),.,.’,.,-()..,,.(...",HRCSL to look into prevailing shortage of medi...,187,"['hrcsl', 'look', 'prevailing', 'shortage', 'm...",hrcsl look prevailing shortage medicine mornin...


In [109]:
# Convert the string to a list of integers
import ast
corpus['preprocessed_article_']  = corpus['preprocessed_article'] .apply(lambda x: ast.literal_eval(x))

In [115]:
# Create a vocabulary mapping words to indices
vocab = {}
index = 0

for article_tokens in corpus['preprocessed_article_'].values:
    for token in article_tokens:
      if token not in vocab:
            vocab[token] = index
            index += 1

In [118]:
# Convert token lists to word indices using the vocabulary
corpus['word_indices'] = corpus['preprocessed_article_'].apply(lambda x : [vocab[token] for token in x ])

In [268]:
#spliting the dataset
dense_train_X, dense_test_X, dense_train_Y, dense_test_Y = model_selection.train_test_split(corpus['word_indices'],corpus['news_source_id'],test_size=0.3)

In [269]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader, Dataset

# Assuming you have a dataset class, adjust this accordingly
class ArticleDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Implement based on your dataset structure
        article = self.data[idx]
        target = self.targets[idx]

        # Pad or trim the article to a fixed length (adjust as needed)
        max_length = 100
        padded_article = article[:max_length] + [0] * (max_length - len(article))

        return torch.tensor(padded_article), torch.tensor(target)

# Simple Transformer model for clustering
class TransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, num_layers):
        super(TransformerModel, self).__init__()
        self.embedding = nn.EmbeddingBag(input_dim, embed_dim)
        self.transformer = nn.Transformer(
            d_model=embed_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers
        )
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, x, offsets):
        x = self.embedding(x, offsets=offsets)
        x = self.transformer(x, x)
        x = self.fc(x)
        return x


In [270]:
# Load GloVe embeddings
glove = GloVe(name='6B', dim =100)

In [271]:
# Assuming you have features and targets as lists
article_features = dense_train_X.tolist()   # Replace with your actual feature data
article_targets = dense_train_Y.tolist()   # Replace with your actual target data

# Create an instance of the custom dataset
article_dataset = ArticleDataset(data=article_features, targets=article_targets)

# You can now use a DataLoader to efficiently iterate over batches
batch_size = 64
train_loader = DataLoader(article_dataset, batch_size=batch_size, shuffle=True)

In [272]:
# Initialize the model
input_dim = len(glove.stoi)
embed_dim = 100  # Adjust based on your GloVe embedding dimension
num_heads = 4
num_layers = 2
model = TransformerModel(input_dim, embed_dim, num_heads, num_layers)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [273]:
import torch.nn.functional as F
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_loader:
        features, targets = batch

        # Assuming features are indices and need to be converted to embeddings
        feature_embeddings = glove[torch.tensor(features, dtype=torch.long)]

        # Calculate offsets based on the lengths of the sequences in the batch
        offsets = torch.tensor([0] + [len(seq) for seq in features], dtype=torch.long).cumsum(dim=0)[:-1]

        optimizer.zero_grad()

        # Pass features and offsets to the model
        outputs = model(feature_embeddings, offsets)
        loss = criterion(outputs.squeeze(), targets)  # Assuming targets are 1D tensor

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/10], Loss: 3.551928758621216
Epoch [2/10], Loss: 3.5381460189819336
Epoch [3/10], Loss: 3.5182275772094727
Epoch [4/10], Loss: 3.71205472946167
Epoch [5/10], Loss: 3.6984167098999023
Epoch [6/10], Loss: 3.5422472953796387
Epoch [7/10], Loss: 3.826603651046753
Epoch [8/10], Loss: 4.0460357666015625
Epoch [9/10], Loss: 2.966336250305176
Epoch [10/10], Loss: 4.84248161315918


In [274]:
# Assuming you have features and targets as lists
article_features_test = dense_test_X.tolist()   # Replace with your actual feature data
article_targets_test= dense_test_Y.tolist()   # Replace with your actual target data

# Create an instance of the custom dataset
val_dataset = ArticleDataset(data=article_features_test, targets=article_targets_test)

# Assuming you have a separate DataLoader for validation or test set
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Set the model to evaluation mode
model.eval()

total_correct = 0
total_samples = 0

In [275]:
# No need to compute gradients during evaluation
with torch.no_grad():
    for batch in val_loader:
        features, targets = batch

        # Assuming you have offsets (lengths of the sequences) for each article
        # Calculate offsets based on the lengths of the sequences
        offsets = torch.tensor([0] + [len(seq) for seq in features], dtype=torch.long).cumsum(dim=0)[:-1]

        # Pass features and offsets to the model
        outputs = model(features, offsets)

        # Get predicted labels
        _, predicted_labels = torch.max(outputs, 1)

        # Update total_correct and total_samples
        total_correct += (predicted_labels == targets).sum().item()
        total_samples += len(targets)

# Calculate accuracy
accuracy = total_correct / total_samples

print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Validation Accuracy: 68.40%
